# W266 Final Project
## Ehsan Yousefzadeh 
## Summer 2018

In [65]:
from __future__ import division
from __future__ import print_function
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

#Tensorflow
from __future__ import print_function
from __future__ import division
import tensorflow as tf

# NLTK 
import nltk

# NumPy, Pandas and TensorFlow
import numpy as np
import pandas as pd
import tensorflow as tf
assert(tf.__version__.startswith("1.8"))
import collections

#SKlearn 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Helper libraries
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst, models, models_test

print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.


# Data Loading and Preparation 

In [66]:
scotch_data = pd.read_csv('./scotch_review.csv')

In [67]:
scotch_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2247 entries, 0 to 2246
Data columns (total 7 columns):
Unnamed: 0      2247 non-null int64
name            2247 non-null object
category        2247 non-null object
review.point    2247 non-null int64
price           2247 non-null object
currency        2247 non-null object
description     2247 non-null object
dtypes: int64(2), object(5)
memory usage: 123.0+ KB


In [68]:
scotch_data

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."
5,6,"Ardbeg Corryvreckan, 57.1%",Single Malt Scotch,96,85.00,$,"Powerful, muscular, well-textured, and invigor..."
6,7,"Gold Bowmore, 1964 vintage, 42.4%",Single Malt Scotch,96,6250.00,$,Deep gold color. Surprisingly lively on the no...
7,8,"Bowmore, 40 year old, 44.8%",Single Malt Scotch,96,11000.00,$,"Definitely showing its age, but not in a bad w..."
8,9,"The Dalmore, 50 year old, 52.8%",Single Malt Scotch,96,1500.00,$,The Dalmore is one of a handful of whiskies th...
9,10,"Glenfarclas Family Casks 1954 Cask #1260, 47.2%",Single Malt Scotch,96,3360,$,A rich amber color and elegantly oxidized note...


In [69]:
scotch_data = scotch_data.drop(scotch_data.index[19])
scotch_data = scotch_data[scotch_data.price != '60,000/set']
scotch_data = scotch_data[scotch_data.price != '44/liter']

In [70]:
scotch_y = scotch_data.price.replace({',':''},regex=True).apply(pd.to_numeric,1)
scotch_y = scotch_y.as_matrix()
scotch_y = scotch_y.astype(float)

In [71]:
np.median(scotch_y)

110.0

In [72]:

#binarize the labels to 0(cheap bottle) or 1(expensive bottle)
scotch_y1 = []

for i in scotch_y:

    if i >= np.median(scotch_y):
        y = 1
    if i< np.median(scotch_y): 
        y = 0
    scotch_y1.append(y)
    
    

In [73]:
scotch_y1

[1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,


In [74]:
scotch_x = scotch_data['description'].as_matrix()

In [75]:
scotch_x[5]

'Powerful, muscular, well-textured, and invigorating. Even within the realm of Ardbeg, this one stands out. The more aggressive notes of coal tar, damp kiln, anise, and smoked seaweed are supported by an array of fruit (black raspberry, black cherry, plum), dark chocolate, espresso, molasses, bacon fat, kalamata olive, and warming cinnamon on the finish. Quite stunning!'

In [76]:
#tokenize the text data 

from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()


scotch_x_tokens = []

for x in scotch_x:
    

    tokens = tokenizer.tokenize(x)
    scotch_x_tokens.append(tokens)


In [77]:
scotch_x_tokens[5]

['Powerful',
 ',',
 'muscular',
 ',',
 'well-textured',
 ',',
 'and',
 'invigorating.',
 'Even',
 'within',
 'the',
 'realm',
 'of',
 'Ardbeg',
 ',',
 'this',
 'one',
 'stands',
 'out.',
 'The',
 'more',
 'aggressive',
 'notes',
 'of',
 'coal',
 'tar',
 ',',
 'damp',
 'kiln',
 ',',
 'anise',
 ',',
 'and',
 'smoked',
 'seaweed',
 'are',
 'supported',
 'by',
 'an',
 'array',
 'of',
 'fruit',
 '(',
 'black',
 'raspberry',
 ',',
 'black',
 'cherry',
 ',',
 'plum',
 ')',
 ',',
 'dark',
 'chocolate',
 ',',
 'espresso',
 ',',
 'molasses',
 ',',
 'bacon',
 'fat',
 ',',
 'kalamata',
 'olive',
 ',',
 'and',
 'warming',
 'cinnamon',
 'on',
 'the',
 'finish.',
 'Quite',
 'stunning',
 '!']

In [78]:
#canonocilize the tokens 
scotch_x_canon = []

for x in scotch_x_tokens: 
    canon = utils.canonicalize_words(x)
    scotch_x_canon.append(canon)


In [79]:
scotch_x_canon[5]

['powerful',
 ',',
 'muscular',
 ',',
 'well-textured',
 ',',
 'and',
 'invigorating.',
 'even',
 'within',
 'the',
 'realm',
 'of',
 'ardbeg',
 ',',
 'this',
 'one',
 'stands',
 'out.',
 'the',
 'more',
 'aggressive',
 'notes',
 'of',
 'coal',
 'tar',
 ',',
 'damp',
 'kiln',
 ',',
 'anise',
 ',',
 'and',
 'smoked',
 'seaweed',
 'are',
 'supported',
 'by',
 'an',
 'array',
 'of',
 'fruit',
 '(',
 'black',
 'raspberry',
 ',',
 'black',
 'cherry',
 ',',
 'plum',
 ')',
 ',',
 'dark',
 'chocolate',
 ',',
 'espresso',
 ',',
 'molasses',
 ',',
 'bacon',
 'fat',
 ',',
 'kalamata',
 'olive',
 ',',
 'and',
 'warming',
 'cinnamon',
 'on',
 'the',
 'finish.',
 'quite',
 'stunning',
 '!']

In [80]:
scotch_flat = []
for x in scotch_x_canon:
    for i in x:
        scotch_flat.append(i)

In [81]:
scotch_flat

['magnificently',
 'powerful',
 'and',
 'intense.',
 'caramels',
 ',',
 'dried',
 'peats',
 ',',
 'elegant',
 'cigar',
 'smoke',
 ',',
 'seeds',
 'scraped',
 'from',
 'vanilla',
 'beans',
 ',',
 'brand',
 'new',
 'pencils',
 ',',
 'peppercorn',
 ',',
 'coriander',
 'seeds',
 ',',
 'and',
 'star',
 'anise',
 'make',
 'for',
 'a',
 'deeply',
 'satisfying',
 'nosing',
 'experience.',
 'silky',
 'caramels',
 ',',
 'bountiful',
 'fruits',
 'of',
 'ripe',
 'peach',
 ',',
 'stewed',
 'apple',
 ',',
 'orange',
 'pith',
 ',',
 'and',
 'pervasive',
 'smoke',
 'with',
 'elements',
 'of',
 'burnt',
 'tobacco.',
 'an',
 'abiding',
 'finish',
 'of',
 'smoke',
 ',',
 'dry',
 'spices',
 ',',
 'and',
 'banoffee',
 'pie',
 'sweetness.',
 'close',
 'to',
 'perfection.',
 'editor',
 "'s",
 'choice',
 'what',
 'impresses',
 'me',
 'most',
 'is',
 'how',
 'this',
 'whisky',
 'evolves',
 ';',
 'it',
 "'s",
 'incredibly',
 'complex.',
 'on',
 'the',
 'nose',
 'and',
 'palate',
 ',',
 'this',
 'is',
 'a',
 'th

In [82]:
vocab = vocabulary.Vocabulary(scotch_flat, size=None)  # size=None means unlimited
print("Vocabulary size: {:,}".format(vocab.size))
# print("Vocabulary dict: ", vocab.word_to_id)

#convert text to ids based on the vocabulary dictionary
scotch_x_ids = []
for x in scotch_x_canon:
    x_ids = vocab.words_to_ids(x)
    scotch_x_ids.append(x_ids)

Vocabulary size: 11,520


In [83]:
vocabulary = vocab.size
vocabulary

11520

In [84]:
scotch_x_ids[5]

[612,
 3,
 1981,
 3,
 4130,
 3,
 4,
 2833,
 191,
 1800,
 5,
 2834,
 7,
 554,
 3,
 11,
 72,
 3353,
 1226,
 5,
 32,
 1227,
 22,
 7,
 555,
 363,
 3,
 252,
 802,
 3,
 406,
 3,
 4,
 325,
 246,
 50,
 4131,
 59,
 37,
 988,
 7,
 30,
 17,
 96,
 350,
 3,
 96,
 292,
 3,
 339,
 18,
 3,
 89,
 47,
 3,
 894,
 3,
 584,
 3,
 1463,
 949,
 3,
 2173,
 614,
 3,
 4,
 420,
 107,
 13,
 5,
 48,
 85,
 1531,
 154]

In [85]:
#Pad the ids so that the vectors are of equal length

max_len = 300
scotch_x_pad = []
scotch_ns = []
for i in scotch_x_ids:
    
    pad_len = max_len - len(i)
    pad_i = np.pad(i, (0, pad_len), 'constant', constant_values =0)
    scotch_ns.append(len(i))
    scotch_x_pad.append(pad_i)

In [86]:
scotch_x_pad[5]

array([ 612,    3, 1981,    3, 4130,    3,    4, 2833,  191, 1800,    5,
       2834,    7,  554,    3,   11,   72, 3353, 1226,    5,   32, 1227,
         22,    7,  555,  363,    3,  252,  802,    3,  406,    3,    4,
        325,  246,   50, 4131,   59,   37,  988,    7,   30,   17,   96,
        350,    3,   96,  292,    3,  339,   18,    3,   89,   47,    3,
        894,    3,  584,    3, 1463,  949,    3, 2173,  614,    3,    4,
        420,  107,   13,    5,   48,   85, 1531,  154,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [87]:
scotch_x_pad_np = np.asarray(scotch_x_pad)
scotch_y_np = np.asarray(scotch_y1)
# scotch_y_np = scotch_y_np.dtype(np.float32)
scotch_ns_np = np.asarray(scotch_ns)

In [88]:
import sst
ds = sst.SSTDataset(V=20000).process(label_scheme="binary")

Loading SST from data/sst/trainDevTestTrees_PTB.zip
Training set:     8,544 trees
Development set:  1,101 trees
Test set:         2,210 trees
Building vocabulary - 16,474 words
Processing to phrases...  Done!
Splits: train / dev / test : 98,794 / 13,142 / 26,052


In [89]:
#split up the data to train, test, and dev sets 

train_x = scotch_x_pad_np[0:1790]
train_ns = scotch_ns_np[0:1790]
train_y = scotch_y_np[0:1790]

test_x = scotch_x_pad_np[1791:2100]
test_ns = scotch_ns_np[1791:2100]
test_y = scotch_y_np[1791:2100]

dev_x = scotch_x_pad_np[2101::]
dev_ns = scotch_ns_np[2101::]
dev_y = scotch_y_np[2101::]


# Base Model (Naive Bayes) Training and Evaluation

In [90]:
#setup the naive bayes model 
nb = MultinomialNB()

nb.fit(train_x, train_y)
y_pred = nb.predict(test_x)

acc = accuracy_score(test_y, y_pred)

print("Accuracy: {:.02f}%".format(100*acc))

Accuracy: 59.22%


# Neural Network Model Training and Evaluation 

In [91]:
def embedding_layer(ids_, V, embed_dim, init_scale=0.001):
   #construct embedding for neural network  
    W_embed_ = tf.get_variable(name = "W_embed", shape = [V, embed_dim],
    initializer = tf.random_uniform_initializer( minval = -init_scale, maxval = init_scale))                 
    xs_ = tf.nn.embedding_lookup(W_embed_, ids_)

    return xs_

def fully_connected_layers(h0_, hidden_dims, activation=tf.tanh,
                           dropout_rate=0, is_training=False):
    
    #create initial and hidden layers for neural network 
    h_ = h0_
    for i, hdim in enumerate(hidden_dims):
        h_ = tf.layers.dense(h_, hdim, activation=activation, name=("Hidden_%d"%i))

        if dropout_rate > 0:
            h_ =  tf.layers.dropout(h_, rate=dropout_rate)

    return h_

def softmax_output_layer(h_, labels_, num_classes):
    #create the output layer for our logits 
    with tf.variable_scope("Logits"):

        W_out_ = tf.get_variable(name = "W_out", shape=[h_.shape[1], num_classes], initializer = tf.random_normal_initializer())
        b_out_ = tf.get_variable("b_out", shape = [num_classes], initializer=tf.zeros_initializer())
        logits_ = tf.add(tf.matmul(h_, W_out_), b_out_) 
        
        #logits = [batch_size for one bottle of whiskey, column for each class (0, 1, etc.)]

        
    if labels_ is None:
        return None, logits_

    with tf.name_scope("Softmax"):
        loss_ = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = labels_, logits = logits_))  
        
        
    return loss_, logits_


def BOW_encoder(ids_, ns_, V, embed_dim, hidden_dims, dropout_rate=0,
                is_training=None,
                **unused_kw):
    assert is_training is not None, "is_training must be explicitly set to True or False"
 
    with tf.variable_scope("Embedding_Layer"):
        xs_ = embedding_layer(ids_, V, embed_dim, init_scale=0.001)

    mask_ = tf.expand_dims(tf.sequence_mask(ns_, xs_.shape[1], dtype=tf.float32),-1)

    sum_e = tf.reduce_sum(xs_*mask_, 1)

    h_ = fully_connected_layers(sum_e, hidden_dims, activation=tf.tanh,
            dropout_rate=dropout_rate, is_training=is_training)

    return h_, xs_


def classifier_model_fn(features, labels, mode, params):
    #create model for running the predictions and regularizations based on the specified 
    tf.set_random_seed(params.get('rseed', 10))

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    if params['encoder_type'] == 'bow':
        with tf.variable_scope("Encoder"):
            h_, xs_ = BOW_encoder(features['ids'], features['ns'],
                                  is_training=is_training,
                                  **params)
    else:
        raise ValueError("Error: unsupported encoder type "
                         "'{:s}'".format(params['encoder_type']))

    with tf.variable_scope("Output_Layer"):
        ce_loss_, logits_ = softmax_output_layer(h_, labels ,params['num_classes'] )

    with tf.name_scope("Prediction"):

        pred_max_ = tf.argmax(logits_, 1, name="pred_max")
        predictions_dict = {"proba": logits_, "max": pred_max_}

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions=predictions_dict)


    with tf.variable_scope("Regularization"):
        l2_penalty_ = tf.nn.l2_loss(xs_)  
        for var_ in tf.trainable_variables():
            if "Embedding_Layer" in var_.name:
                continue
            l2_penalty_ += tf.nn.l2_loss(var_)
        l2_penalty_ *= params['beta']  
        tf.summary.scalar("l2_penalty", l2_penalty_)
        regularized_loss_ = ce_loss_ + l2_penalty_

    with tf.variable_scope("Training"):
        if params['optimizer'] == 'adagrad':
            optimizer_ = tf.train.AdagradOptimizer(params['lr'])
        else:
            optimizer_ = tf.train.GradientDescentOptimizer(params['lr'])
        train_op_ = optimizer_.minimize(regularized_loss_,
                                            global_step=tf.train.get_global_step())

    tf.summary.scalar("cross_entropy_loss", ce_loss_)
    eval_metrics = {"cross_entropy_loss": tf.metrics.mean(ce_loss_),
                    "accuracy": tf.metrics.accuracy(labels, pred_max_)}

    return tf.estimator.EstimatorSpec(mode=mode,
                                      predictions=predictions_dict,
                                      loss=regularized_loss_,
                                      train_op=train_op_,
                                      eval_metric_ops=eval_metrics)


In [92]:

#specify the model parameters for training
model_params = dict(V=vocabulary, embed_dim=50, hidden_dims=[25], num_classes = 2,
                    encoder_type='bow',
                    lr=0.1, optimizer='adagrad', beta=0.01)

checkpoint_dir = "/tmp/tf_bow_sst_" + datetime.datetime.now().strftime("%Y%m%d-%H%M")
if os.path.isdir(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)

ds.vocab.write_projector_config(checkpoint_dir, "Encoder/Embedding_Layer/W_embed")

model = tf.estimator.Estimator(model_fn= classifier_model_fn, 
                               params=model_params,
                               model_dir=checkpoint_dir)
print("")
print("To view training (once it starts), run:\n")
print("    tensorboard --logdir='{:s}' --port 6006".format(checkpoint_dir))
print("\nThen in your browser, open: http://localhost:6006")

Vocabulary (16,474 words) written to '/tmp/tf_bow_sst_20180807-2346/metadata.tsv'
Projector config written to /tmp/tf_bow_sst_20180807-2346/projector_config.pbtxt
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tf_bow_sst_20180807-2346', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8707d3e940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}

To view training (once it starts), run:

    tensorboard --logdir='/tmp/tf_bow_sst_20180807-2346' --port 6006

Then in your browser, open: http://localhost:6006


In [93]:
train_params = dict(batch_size=32, total_epochs=20, eval_every=2)
assert(train_params['total_epochs'] % train_params['eval_every'] == 0)

train_input_fn = patched_numpy_io.numpy_input_fn(
                    x={"ids": train_x, "ns": train_ns}, y=train_y,
                    batch_size=train_params['batch_size'], 
                    num_epochs=train_params['eval_every'], shuffle=True, seed=42
                 )

dev_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": dev_x, "ns": dev_ns}, y=dev_y,
                    batch_size=128, num_epochs=1, shuffle=False
                )

for _ in range(train_params['total_epochs'] // train_params['eval_every']):

    model.train(input_fn=train_input_fn)
    eval_metrics = model.evaluate(input_fn=dev_input_fn, name="dev")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tf_bow_sst_20180807-2346/model.ckpt.
INFO:tensorflow:loss = 1.2027862, step = 1
INFO:tensorflow:global_step/sec: 121.625
INFO:tensorflow:loss = 1.09442, step = 101 (0.823 sec)
INFO:tensorflow:Saving checkpoints for 112 into /tmp/tf_bow_sst_20180807-2346/model.ckpt.
INFO:tensorflow:Loss for final step: 1.2127151.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-07-23:46:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180807-2346/model.ckpt-112
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-07-23:46:22


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-07-23:46:34
INFO:tensorflow:Saving dict for global step 784: accuracy = 0.74285716, cross_entropy_loss = 0.5651026, global_step = 784, loss = 0.88223094
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180807-2346/model.ckpt-784
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 785 into /tmp/tf_bow_sst_20180807-2346/model.ckpt.
INFO:tensorflow:loss = 0.2760799, step = 785
INFO:tensorflow:global_step/sec: 124.493
INFO:tensorflow:loss = 0.2545421, step = 885 (0.804 sec)
INFO:tensorflow:Saving checkpoints for 896 into /tmp/tf_bow_sst_20180807-2346/model.ckpt.
INFO:tensorflow:Loss for final step: 0.24729171.
INFO:tensorflow:Calli

In [94]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
                    x={"ids": test_x, "ns": test_ns}, y=test_y,
                    batch_size=128, num_epochs=1, shuffle=False)

eval_metrics = model.evaluate(input_fn=test_input_fn, name="test")  # replace with result of model.evaluate(...)
print("Accuracy on test set: {:.02%}".format(eval_metrics['accuracy']))
eval_metrics

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-07-23:46:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180807-2346/model.ckpt-1120
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-07-23:46:42
INFO:tensorflow:Saving dict for global step 1120: accuracy = 0.6666667, cross_entropy_loss = 0.88387376, global_step = 1120, loss = 1.1451517
Accuracy on test set: 66.67%


{'accuracy': 0.6666667,
 'cross_entropy_loss': 0.88387376,
 'global_step': 1120,
 'loss': 1.1451517}

In [95]:
from sklearn.metrics import accuracy_score
predictions = list(model.predict(test_input_fn))  # list of dicts
y_pred = [p['max'] for p in predictions]
acc = accuracy_score(y_pred, test_y)
print("Accuracy on test set: {:.02%}".format(acc))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tf_bow_sst_20180807-2346/model.ckpt-1120
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Accuracy on test set: 66.67%


In [96]:
test_y

array([0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,

In [97]:
y_pred

[0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,


In [98]:
#Take a look at the classification report 
print("Classification Report: \n", classification_report(test_y, y_pred))
print("Confusion Matrix: \n",confusion_matrix(test_y, y_pred))

Classification Report: 
              precision    recall  f1-score   support

          0       0.81      0.65      0.72       204
          1       0.51      0.70      0.59       105

avg / total       0.71      0.67      0.68       309

Confusion Matrix: 
 [[132  72]
 [ 31  74]]


In [99]:
misclassified = np.where(test_y != y_pred)
classified = np.where(test_y == y_pred)

In [100]:
misclassified

(array([  1,   2,   4,   5,  13,  14,  15,  20,  21,  22,  24,  25,  36,
         40,  46,  49,  51,  53,  63,  64,  67,  69,  73,  77,  78,  81,
         86,  90,  91,  93,  94,  98,  99, 106, 107, 109, 111, 112, 114,
        119, 121, 123, 125, 130, 132, 137, 144, 145, 147, 148, 153, 158,
        159, 162, 163, 166, 167, 168, 169, 175, 178, 179, 180, 181, 182,
        185, 186, 190, 193, 202, 205, 209, 210, 213, 215, 219, 231, 232,
        234, 235, 236, 239, 241, 250, 253, 254, 261, 262, 264, 266, 267,
        271, 274, 277, 286, 287, 289, 292, 295, 299, 301, 303, 304]),)

In [101]:
classified

(array([  0,   3,   6,   7,   8,   9,  10,  11,  12,  16,  17,  18,  19,
         23,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  37,  38,
         39,  41,  42,  43,  44,  45,  47,  48,  50,  52,  54,  55,  56,
         57,  58,  59,  60,  61,  62,  65,  66,  68,  70,  71,  72,  74,
         75,  76,  79,  80,  82,  83,  84,  85,  87,  88,  89,  92,  95,
         96,  97, 100, 101, 102, 103, 104, 105, 108, 110, 113, 115, 116,
        117, 118, 120, 122, 124, 126, 127, 128, 129, 131, 133, 134, 135,
        136, 138, 139, 140, 141, 142, 143, 146, 149, 150, 151, 152, 154,
        155, 156, 157, 160, 161, 164, 165, 170, 171, 172, 173, 174, 176,
        177, 183, 184, 187, 188, 189, 191, 192, 194, 195, 196, 197, 198,
        199, 200, 201, 203, 204, 206, 207, 208, 211, 212, 214, 216, 217,
        218, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 233,
        237, 238, 240, 242, 243, 244, 245, 246, 247, 248, 249, 251, 252,
        255, 256, 257, 258, 259, 260, 263, 265, 268

In [102]:
samples_x = scotch_x[1791:2100]
samples_y = scotch_y1[1791:2100]

In [103]:
for i in misclassified[0][0:20]:
    if samples_y[i] == 1:
        print("Misclassified as 0 that should be a 1:\n",samples_x[i])
    

Misclassified as 0 that should be a 1:
 Lovely golden honey color. Lush and sweet (the Sauternes impact is obvious), with honey-drenched apricot, sultana, and lemon gum drops. Vanilla, candied nuts, and subtle botanicals round out the palate. Decent oak grip on the finish keeps the whisky from being too cloying.

Misclassified as 0 that should be a 1:
 The only distillery-approved bottling of Bladnoch in the United States. When compared to the Gordon & MacPhail bottling above, this one isn’t as sweet or creamy in texture. There’s more citrus fruit and a drier, spicier finish. This one is also bolder and a bit more aggressive. 

Misclassified as 0 that should be a 1:
 Aged for six years in bourbon casks, then aged an additional ten years in Bordeaux wine casks. Classic Bowmore peat smoke and red berried fruit (raspberry, strawberry) are the main components of this whisky, with additional notes of plum, grape skin tannins, currant, caramel, and marzipan. Good grip from the Limousin oak (

In [104]:
for i in misclassified[0][0:20]:
    if samples_y[i] == 0:
        print("Misclassified as 1 that should be a 0: \n",samples_x[i])

Misclassified as 1 that should be a 0: 
 In the same genre as BenRiach 12 year old reviewed below, but more concentrated, with additional notes of resinous oak, light toffee, and soft caramelized pear. 

Misclassified as 1 that should be a 0: 
 The second bottling from Bruichladdich in this range shows the ‘traditional‘ side of a distillery that is famed for the variety of its expressions. This, in other words, is Bruichladdich in its sweet, honeyed, gentle guise partnered with American oak to produce a mashed banana character, an unctuous palate, and a light touch of almond on the finish. Lovely. £55 (500 ml)
Misclassified as 1 that should be a 0: 
 As this is slightly closed when neat, you have to search for Glen Grant’s classic apple notes behind a tense nose which, even with water, remains inward-looking. Thankfully, this changes on the tongue with pure, very linear (typical of GG) flavors of cool mint, cucumber, basil, and yes, apple. Easy-going from a relaxed cask, but with more 